In [11]:
import torch.nn as nn
from transformers import BertModel
import torch
from transformers import AdamW
from tqdm import tqdm
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

In [12]:
# Load tokenized dataset
data = torch.load('../models/tokenized_dataset.pt')

input_ids = data['input_ids']
attention_masks = data['attention_masks']
features_tensor = data['features_tensor']
labels = data['labels']

C:\Users\taha_kms\AppData\Local\Temp\ipykernel_17624\2330390741.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load('../models/tokenized_dataset.pt')


In [13]:
dataset = TensorDataset(input_ids, attention_masks, features_tensor, labels)

batch_size = 16
dataloader = DataLoader(dataset, sampler=RandomSampler(dataset), batch_size=batch_size)


### Define the BERT + Tabular Model

In [14]:

class BERTWithTabular(nn.Module):
    def __init__(self):
        super(BERTWithTabular, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.tabular_layer = nn.Linear(3, 32)  # 3 features: hour, weekday, emoji
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(768 + 32, 2)  # BERT output + tabular

    def forward(self, input_ids, attention_mask, tabular_input):
        bert_out = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = bert_out.pooler_output
        tabular_output = self.tabular_layer(tabular_input)
        combined = torch.cat((pooled_output, tabular_output), dim=1)
        combined = self.dropout(combined)
        logits = self.classifier(combined)
        return logits

### Define Optimizer, Loss, Device

In [15]:
epochs = 3  # Or whatever you want
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = BERTWithTabular().to(device)
optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = nn.CrossEntropyLoss()

In [16]:
model.train()
for epoch in range(epochs):
    total_loss = 0
    for batch in tqdm(dataloader):
        input_ids, attention_mask, tabular_features, labels = [b.to(device) for b in batch]

        outputs = model(input_ids, attention_mask, tabular_features)
        loss = loss_fn(outputs, labels)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()

    print(f"Epoch {epoch+1} | Loss: {total_loss/len(dataloader):.4f}")


  0%|          | 0/100000 [00:04<?, ?it/s]


IndexError: Target 4 is out of bounds.